import laibraris

In [617]:
import nltk

import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import os

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import SVC
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from datetime import date
nltk.download('punkt')
import pandas
import csv
from numpy import vectorize
from pandas import DataFrame
from pandas.io.parsers import TextFileReader
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors.kde import KernelDensity
import random
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from scipy.sparse import dia_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import pandas as pd



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abughosh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading the data 

In [618]:
results_df = pd.DataFrame(columns=["id","label"])
path = "../Detecting-Cyber-Attacks-Kaggle-Callenge/data/"

In [698]:

#read labeld data for user i
user_test_f={}
user_train_f={}
for testUser in range(10,40):
    #read labeld data
    dataset = pandas.read_csv(path+"partial_labels.csv")
    #for all
    x,segments=dataset.axes
    array=dataset.values

    #read instructions
    fileName =array[testUser,0]
    path_read =path + "FraudedRawData/User"+str(testUser)
    uset_data = pd.read_csv(path_read, sep=" ", header=None)
    arr=np.reshape(uset_data.values,(150,100))
    user_test_f[testUser-10]=np.array(arr[0:,:])
    user_train_f[testUser-10]=arr[:50,:]
    masq=[]

    for i in range(10):
        partial_label = np.reshape(array[i, 1:], -1)
        partial_label = partial_label.astype('int')
        fileName = array[i, 0]
        path_read =path + "FraudedRawData/User"+str(i)
        uset_data = pd.read_csv(path_read, sep=" ", header=None)
        arr = np.reshape(uset_data.values, (150, 100))
        for j in range(150):
            if(partial_label[j]==1):
                masq.append(arr[j,:])

In [699]:
user_train

array([['cpp', 'sh', 'xrdb', ..., 'ls', 'tcsh', 'cat'],
       ['stream_t', 'call_fil', 'overlap', ..., 'tcsh', 'cat', 'grep'],
       ['sed', 'tcsh', 'cat', ..., 'sed', 'tcsh', 'list2.pl'],
       ...,
       ['grep', 'tcsh', 'sort', ..., 'sed', 'tcsh', 'cat'],
       ['grep', 'sed', 'tcsh', ..., 'cat', 'grep', 'sed'],
       ['tcsh', 'list2.pl', 'ls', ..., 'tcsh', 'sort', 'sh']],
      dtype=object)

In [700]:
masq=np.array(masq)
t={}
for i in range(10,40):
    user_train=np.array(user_train_f[i-10])
    usr_masq=np.concatenate((user_train,masq))

    train_user_data=[]
    corpus=[]
    test_corpus=[]
    
    
        ##################################################################
    wordsCount={}
    uniqe=set()
    unique_feature_train=np.zeros(150)
    unique_feature_test=np.zeros(150)
    for z in range(len(usr_masq)):
        for j in range(100):
            if(z<50):
                uniqe.add(usr_masq[z][j])
                unique_feature_train[z]=random.randint(0, 7)
            else:
                if (usr_masq[z][j] not in uniqe):
                    unique_feature_train[z] = random.randint(12, 20)

    for z in range(len(user_test_f[i-10])):
        if(z<50):
            unique_feature_test[z]=random.randint(0,10)
        for j in range(100):
               if (user_test_f[i-10][z][j] not in uniqe):
                      unique_feature_test[z] = unique_feature_test[z] + 1




    ##################################################################

    for j in range(len(user_test_f[i-10])):
        s=' '.join((user_test_f[i-10][j,:]))
        test_corpus.append(s)


    for j in range(len(usr_masq)):
        s=' '.join((usr_masq[j,:]))
        if(j<50):
          train_user_data.append(s)
          corpus.append(s)
        else:
          corpus.append(s)



    if(i==35):
        vectorizer=CountVectorizer(ngram_range=(2, 2),min_df=0.4)
    else:
        vectorizer=CountVectorizer(ngram_range=(2, 2),min_df=0.5)
    vectorizer.fit_transform(train_user_data)
    train_all=vectorizer.transform(corpus)
    x_test=vectorizer.transform(test_corpus)






    train_all=np.array(train_all.toarray())
    x_test=np.array(x_test.toarray())
    classes=np.concatenate((np.zeros(50),np.ones(100)))

    train_all=np.c_[ train_all, unique_feature_train ]
    x_test=np.c_[x_test,unique_feature_test]

    for z in range(len(train_all)):
        print(train_all[z,:])


    clf = SVC(gamma='auto',kernel='linear')
    clf.fit(train_all,classes)

    predicts=clf.predict(x_test)


    t[i-10]=clf.predict(x_test)
    t[i-10]=t[i-10].astype(int)



 



[ 2.  2.  0.  1.  2.  5.  6.  2.  3. 26.  1.  2.  2.  2.  0.]
[5. 5. 3. 1. 5. 1. 2. 6. 3. 1. 2. 0. 1. 4. 0.]
[ 1.  1.  0.  1.  1.  3.  5.  1.  1. 11.  0.  2.  3.  1.  7.]
[1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 7.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6.]
[3. 3. 5. 0. 2. 0. 0. 3. 8. 0. 1. 5. 0. 2. 5.]
[ 3.  3.  4.  1.  3.  2.  5.  3. 13.  1.  0.  9.  5.  4.  3.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 8. 1. 5. 0. 0. 4.]
[ 1.  1.  0.  1.  1.  1.  2.  1. 18. 21.  2.  5.  1.  1.  1.]
[3. 3. 3. 1. 3. 4. 4. 3. 3. 0. 2. 0. 2. 3. 7.]
[4. 4. 5. 0. 4. 0. 0. 4. 6. 0. 1. 9. 0. 4. 3.]
[ 3.  3.  1.  2.  3.  5.  6.  3.  1. 19.  4.  4.  3.  3.  1.]
[4. 4. 2. 1. 4. 4. 4. 5. 2. 0. 0. 4. 2. 4. 1.]
[ 0.  0.  1.  0.  0.  0.  2.  0.  2. 12.  0.  2.  1.  0.  4.]
[1. 1. 0. 1. 1. 1. 2. 1. 8. 7. 0. 4. 1. 1. 5.]
[4. 4. 4. 1. 4. 4. 4. 4. 3. 0. 1. 2. 1. 4. 6.]
[ 2.  2.  0.  2. 

[2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 2.
 0. 0. 2. 0. 0. 2. 3. 1.]
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 3. 4. 0. 0. 0. 0. 0. 0. 0. 3.
 0. 1. 3. 0. 0. 2. 1. 3.]
[2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.
 0. 0. 3. 0. 0. 2. 3. 1.]
[2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 2.
 0. 1. 2. 0. 0. 2. 3. 7.]
[2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 0. 3. 0. 0. 2. 2. 0.]
[3. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 7.
 0. 0. 3. 0. 0. 4. 5. 1.]
[ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  7. 13.  0.  0.
  0.  0.  0.  0.  0.  2.  0.  9.  1.  0.  0.  1.  1.  6.]
[4. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 1. 4. 0. 0. 4. 5. 4.]
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 5. 8. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 2. 1. 0. 0. 1. 1. 6.]
[3. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 5.
 0. 

[ 0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  1.  1.
  1.  0.  2.  1.  1.  0.  2.  0.  0.  1.  1.  0.  0. 15.]
[ 0.  3.  0.  3.  3.  0.  3.  2.  3.  2.  3.  0.  3.  3.  0.  0.  2.  3.
  3.  0.  1.  3.  3.  0.  1.  0.  0.  3.  3.  0.  0. 19.]
[ 0.  1.  0.  1.  1.  0.  1.  2.  1.  2.  1.  0.  1.  1.  0.  0.  1.  1.
  1.  0.  3.  1.  1.  0.  2.  0.  0.  1.  1.  0.  0. 16.]
[ 0.  2.  0.  2.  2.  0.  2.  1.  2.  1.  2.  0.  2.  2.  0.  0.  2.  2.
  2.  0.  2.  2.  2.  0.  2.  0.  0.  2.  2.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  2.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  1.  1.
  1.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  0. 13.]
[ 0.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  0.  0.  1.  1.  1.
  1.  1.  2.  0.  1.  0.  2.  1.  0.  1.  1.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  3.  0.  0.
  0.  0.  

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 15.]
[ 0.  0.  1.  2.  0.  0.  2.  3.  0.  2.  3.  0.  0.  4.  1.  3.  2.  1.
  0.  4.  1.  0.  0.  3.  0.  0.  3.  0.  3.  0.  0.  1.  2.  3.  1.  4.
  3.  0.  4.  0.  0.  0.  2.  3.  0.  0.  0.  0.  3.  0.  0.  0.  0. 15.]
[ 0.  0.  1.  0.  0.  0.  4.  1.  0.  0.  2.  0.  0.  1.  3.  2.  0.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  4.
  3.  0.  0.  0.  0.  0.  0.  3.  0.  0.  1.  0.  1.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.  2.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  1.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0.  1.  0.  0.  0. 13.]
[ 0.  0.  1.  1.  0.  0.  2.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  1.
  0.  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  1.  1.  1.  1.  0.
  1.  1.  1.  0.  0.  0.  1.  1.  0.  2.  0.  0.  1.  0.  0.  0.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

...

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  3.  1.  0.  0.  0.  0.  0. 15.]
[ 1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0. 19.]
[ 3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 3.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0. 15.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 1.  0.  

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.  0.  0.  0.  3.  0.  0.  3.  3.  0.  3.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 2.  0.  0.  2.  0.  0.  0. 11.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 2.  1.  0.  2.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 1.  0.  0.  1.  0.  0.  0.  9.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0. 12.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 3.  1.  0.  3.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. 13.]
[ 1.  0.  0.  1.  0.  0.  0.  7.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. 17.]
[ 0.  0.  0.  0.  0.  1.  

[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 15.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 15.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  3.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  

[0. 3. 6. 0. 4. 5.]
[0. 2. 4. 0. 3. 3.]
[1. 1. 3. 3. 3. 3.]
[2. 1. 2. 0. 1. 5.]
[ 1.  0.  3. 10.  3.  6.]
[ 0.  3.  3. 14.  1.  1.]
[ 0.  0.  3. 31.  2.  4.]
[1. 0. 0. 0. 0. 3.]
[1. 2. 2. 0. 2. 3.]
[2. 2. 2. 0. 0. 6.]
[1. 1. 1. 5. 1. 0.]
[0. 0. 0. 0. 0. 5.]
[2. 0. 1. 8. 1. 4.]
[2. 2. 2. 8. 1. 6.]
[ 1.  1.  3. 42.  1.  1.]
[ 3.  2.  3. 14.  1.  3.]
[ 1.  0.  1. 23.  0.  1.]
[4. 1. 7. 0. 5. 1.]
[3. 0. 6. 0. 3. 2.]
[ 0.  1.  5. 19.  2.  0.]
[ 1.  0.  0. 21.  0.  4.]
[2. 1. 3. 1. 2. 3.]
[ 0.  1.  1. 62.  1.  5.]
[0. 0. 0. 5. 0. 5.]
[0. 0. 0. 0. 0. 5.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 5.]
[ 6.  1.  1. 12.  0.  3.]
[10.  0.  1. 15.  1.  1.]
[ 0.  0.  0. 10.  0.  2.]
[ 3.  1.  1. 13.  1.  0.]
[ 2.  1.  1. 18.  0.  2.]
[ 4.  0.  0. 34.  0.  3.]
[ 0.  2.  3. 37.  3.  6.]
[ 2.  0.  0. 32.  0.  4.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 2. 0. 3.]
[ 0.  2.  3. 10.  0.  1.]
[1. 0. 0. 5. 0. 1.]
[ 1.  0.  1. 26.  1.  1.]
[ 3.  1.  2. 12.  1.  4.]
[ 2.  1.  1. 12.  1.  0.]
[ 0.  1.  0. 1

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. 15.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0. 19.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0. 20.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 14.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 13.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 18.]
[ 0.  0.  0.  

In [701]:
results_df = pd.DataFrame(columns=["id","label"])
for i in range(10,40):
    curr_user_results = t[i-10][50:150]
    curr_user_results_df = pd.DataFrame(columns=["id","label"])

    start_rows = 5000
    till_rows = 5100

    for j in range(curr_user_results.size):
        curr_user_results_df.loc[j] = ["User{}_{}-{}".format(i,start_rows,till_rows),curr_user_results[j]]
        start_rows = start_rows + 100
        till_rows = till_rows + 100
    results_df = results_df.append(curr_user_results_df)

In [667]:
fname = "out/submission_results_{}_.csv".format(date.today())
results_df.to_csv(fname, index=False)

In [681]:
t

{0: array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1

In [702]:
results_df 

,id,label
0,User10_5000-5100,1
1,User10_5100-5200,0
2,User10_5200-5300,0
3,User10_5300-5400,1
4,User10_5400-5500,0
5,User10_5500-5600,0
6,User10_5600-5700,1
7,User10_5700-5800,0
8,User10_5800-5900,0
9,User10_5900-6000,0
